## Sets:
- J = [1:j] -> nodes of demand points(DP)
- F = [1:f] -> product families(PF)
- T = [1:t] -> planning horizont 
- I = [1:i] -> nodes of manufacturing plants(MP) 
- K = [1,k] -> nodes of storage facilities(SF) 
- st = ['Large', 'Medium', 'Small'] -> storage type/Size 

## Multidimentional Sets:
- FJ=[f,j] -> Product Families requiers by each Demand Point  
- FT=[f,st] -> Product Families able to locate in each a st facility  
- TK=[st,k] -> Potencial tipes st allowed to instal in node k 

## Parameters:
- D[j,f,t] -> annual demand of a product in an specific demand point  
- av[f,i]  -> annual availability of a MP 
- q_lo[f,st] -> Minimum amount of a PF f in a storage facility of type st -> 
- q_up[f,st] -> Maximum amount of a PF f in a a storage facility of type st -> 
- tq_lo[st] -> Global minimum of type st 
- tq_up[st] -> Global maximum of type st
### Costs and Investments
- cp[f,i] -> cost of purchase 
- ffmc[f] -> mantainance fost of a PF
- fmc[st] -> fixed cost of a SF of type 'st'
- vhc[f,st] -> operational handling cost
- Inv[st] -> Investment cost of SF type st
- FInv[f,st] -> Investment for alocating a PF 'f' in a SF of type 'st'.

## Binary Variables:
- v[k] -> The node k is adopted
- w[k,st] -> The node k is adopted with a storage capacity st
- wq[f,k,st] -> A PF 'f'f is managed in the SF 'k' of type 'st'
- q[f,k] -> A PF 'f' is managed in the SF 'k'

- x[f,i,k,t] -> Primary Links
- y[f,k,kk,t] -> Intermediate Links
- z[f,k,j,t] -> Finals Links

## Positive Variables: 
- QFC[f, k, j, t] -> Annual flow of a PD 'f' from a SF 'k' to a DP 'j'
- QFM[f, i, k, t] -> Annual purchased of 'f' from 'i' and shiped to 'k'
- QFK[f, k, kk, t] ->  Annual flow of 'f' from 'k' to 'kk'
- QTF[f, k, t] ->  Annual flow of 'f' moving across 'k'
- TQ[k,t] -> Global annual flow acorss 'k'
- QTTF[f,k,st,t]
### Cost Variables
- Capex[k] -> Total Capex of SF in 'k'
- FFInv[f,k,st] -> Product Alocation Investment
- TI -> Total Invesments
- TPC[t] -> Total Purchase Cost
- Tfmc[t] -> Total Fixed Cost
- Opex[k,t] -> Operational Cost of ST in 'k'
- TOC[t] -> Total Operational Handling Cost
- TTC1 -> Primary transport Cost
- TTC2 -> Intermediate transport Cost
- TTC3 -> Final transport Cost
- TTC -> Total Transport Cost

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cplex
import pandas as pd
import pyomo.environ as pyo

In [2]:
base_url = 'https://raw.githubusercontent.com/vitostamatti/mathematical-optimization-pyomo/main/data/06-supply_chain_design-03'

url_data = base_url + '/inputs.xlsx'

In [3]:
# df_sets = pd.read_excel('../datos/inputs_supply_chain.xlsx', sheet_name="SETS")
df_sets = pd.read_excel(url_data, sheet_name="SETS")

In [4]:
T = list(df_sets.year.dropna(axis=0))
P = list(df_sets.products.dropna(axis=0))
ST = list(df_sets.facility_type.dropna(axis=0))
I = list(df_sets.nodes_i.dropna(axis=0))
K = list(df_sets.nodes_k.dropna(axis=0))
J = list(df_sets.nodes_j.dropna(axis=0))

In [5]:
# dem = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="DEMAND")
dem = pd.read_excel(url_data, sheet_name="DEMAND")
dem.set_index(['J','P','T'], inplace=True)

In [6]:
dem.head()

demand  rand_demand
J  P  T                      
j0 p0 t0      84           79
      t1      72           32
      t2      66           26
      t3       9           93
      t4      24           62

In [7]:
# df_nodes = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="NODES")
df_nodes = pd.read_excel(url_data, sheet_name="NODES")

In [8]:
nodes_i = df_nodes[['node_i','x_i','y_i']].copy()
nodes_i.dropna(axis=0,inplace=True)
nodes_i.set_index('node_i', inplace=True)

nodes_k = df_nodes[['node_k','x_k','y_k']].copy()
nodes_k.dropna(axis=0,inplace=True)
nodes_k.set_index('node_k', inplace=True)

nodes_j = df_nodes[['node_j','x_j','y_j']].copy()
nodes_j.dropna(axis=0,inplace=True)
nodes_j.set_index('node_j', inplace=True)

In [9]:
dist_1 = {}
for i in I:
    for k in K:
        dist_1[(i,k)] = np.sqrt( (nodes_i.loc[i,'x_i']-nodes_k.loc[k,'x_k'])**2 +  (nodes_i.loc[i,'y_i']-nodes_k.loc[k,'y_k'])**2)/1000
        
dist_2 = {}
for k in K:
    for kk in K:
        dist_2[(k,kk)] = np.sqrt( (nodes_k.loc[k,'x_k']-nodes_k.loc[kk,'x_k'])**2 +  (nodes_k.loc[k,'y_k']-nodes_k.loc[kk,'y_k'])**2)/1000
        
dist_3 = {}
for k in K:
    for j in J:
        dist_3[(k,j)] = np.sqrt( (nodes_k.loc[k,'x_k']-nodes_j.loc[j,'x_j'])**2 +  (nodes_k.loc[k,'y_k']-nodes_j.loc[j,'y_j'])**2)/1000

In [10]:
# df_facilities_1 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="FACILITIES_1")
df_facilities_1 = pd.read_excel(url_data, sheet_name="FACILITIES_1")
df_facilities_1.set_index('ST',inplace=True)

In [11]:
df_facilities_1

,low,up,capex,fixed_cost
ST,,,,
st0,0,100000,10,1.000000
st1,0,100000,12,1.095445
st2,0,100000,15,1.224745


In [12]:
# df_facilities_2 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="FACILITIES_2")
df_facilities_2 = pd.read_excel(url_data, sheet_name="FACILITIES_2")
df_facilities_2.set_index(['ST','P'], inplace=True)

In [13]:
df_facilities_2.head()

low   up  rand_low  rand_up  allocation_cost  allocation_rand  \
ST  P                                                                   
st0 p0    0  500         0       30         0.021442         0.002748   
    p1    0  500        20       28         0.053293         0.098812   
    p2    0  500        11       34         0.008051         0.004416   
    p3    0  500         5       31         0.067314         0.040627   
    p4    0  500        19       36         0.057761         0.055251   

        handling_cost  processing_time  up_2  low_2  
ST  P                                                
st0 p0       0.000043              0.5    40      2  
    p1       0.000107              0.5    23      4  
    p2       0.000016              0.5    20      5  
    p3       0.000135              0.5    29      5  
    p4       0.000116              0.5    28      7

In [14]:
# df_suppliers_1 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="SUPPLIERS_1")
df_suppliers_1 = pd.read_excel(url_data, sheet_name="SUPPLIERS_1")
df_suppliers_1.set_index('I',inplace=True)

In [15]:
df_suppliers_1

,up
I,
i0,99999
i1,99999
i2,99999
i3,99999


In [16]:
# df_suppliers_2 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="SUPPLIERS_2")
df_suppliers_2 = pd.read_excel(url_data, sheet_name="SUPPLIERS_2")
df_suppliers_2.set_index(['I','P'],inplace=True)

In [17]:
df_suppliers_2.head()

up  rand_up  purchase_cost  rand_p_cost  up_2
I  P                                                   
i0 p0  99999      224       0.002557     0.007749   295
   p1  99999      425       0.004364     0.005555   283
   p2  99999      426       0.004604     0.009635   432
   p3  99999      146       0.004101     0.008924    40
   p4  99999      412       0.008695     0.009195   455

In [18]:
# df_products = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="PRODUCTS")
df_products = pd.read_excel(url_data, sheet_name="PRODUCTS")
df_products.set_index('P',inplace=True)

In [19]:
df_products.head()

,t_primary_cost,t_intermediate_cost,t_final_cost,inventory_cost,rand_inv_cost,waiting_cost,rand_waiting_cost,mantainance_cost,rand_mant_cost
P,,,,,,,,,
p0,0.01,0.05,0.1,0.000889,0.003165,0.004133,0.005769,0.005511,0.006044
p1,0.01,0.05,0.1,0.009633,0.005849,0.005742,0.005990,0.000723,0.007004
p2,0.01,0.05,0.1,0.000127,0.001763,0.005728,0.003925,0.001833,0.008877
p3,0.01,0.05,0.1,0.008537,0.009023,0.001690,0.006232,0.000871,0.007218
p4,0.01,0.05,0.1,0.006305,0.009240,0.000504,0.009113,0.007173,0.000501


In [20]:
def build_model():
    #SETS
    m = pyo.ConcreteModel()
    
    m.J = pyo.Set(initialize = J, doc = "Demand Points")
    m.P = pyo.Set(initialize = P, doc = "Products")
    m.T = pyo.Set(initialize = T ,doc = "Planning Horizont", ordered=True)
#     m.N = Set(initialize =  ,doc = "Nodes of Network")
    
    m.I = pyo.Set(initialize=I, doc = "Nodes of Manufacturing Plants")
    m.K = pyo.Set(initialize=K, doc = "Nodes of Storage Facilities")
    
    m.ST = pyo.Set(initialize=ST, doc = "Type of Storage Facilities")
    
    m.PJ = pyo.Set(m.P,m.J, doc = "Products requires by each demand point")
    m.PST = pyo.Set(m.P, m.ST, doc = "Products able to locate in each facility type")
    m.KST = pyo.Set(m.K, m.ST, doc = "Facility type allowed to locate in each node")

    big_m = 1000000
    
    #VARIABLES
    m.v = pyo.Var(m.K, domain=pyo.Binary, doc = "The node k is adopted", initialize=0)
    m.w = pyo.Var(m.K, m.ST, domain=pyo.Binary, doc = "The node k is adopted with a facility type st", initialize=0 )
    m.wq = pyo.Var(m.K, m.P, m.ST, domain=pyo.Binary, doc="A product 'p'f is managed in the node 'k' of type 'st'",  initialize=0)
    m.q = pyo.Var(m.K, m.P, domain=pyo.Binary ,doc="A product p is manage in the node k", initialize=0 )
    
    #Link variables
    m.x = pyo.Var(m.I,m.K,m.P,m.T, domain=pyo.Binary, doc="Primary links" , initialize=0)
    m.y = pyo.Var(m.K,m.K,m.P,m.T, domain=pyo.Binary, doc="Intermediate links", initialize=0 )
    m.z = pyo.Var(m.K,m.J,m.P,m.T, domain=pyo.Binary, doc="Final links" , initialize=0)
    
    m.QIK = pyo.Var(m.I, m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    m.QKJ = pyo.Var(m.K, m.J, m.P, m.T, domain=pyo.NonNegativeReals)
    m.QKK = pyo.Var(m.K, m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    
    m.QKST = pyo.Var(m.K, m.P, m.ST, m.T, domain=pyo.NonNegativeReals)
    
    m.QK = pyo.Var(m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    m.QK_total = pyo.Var(m.K, m.T, domain=pyo.NonNegativeReals)
    
    m.flow_in = pyo.Var(m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    m.flow_out = pyo.Var(m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    
    m.capex_products = pyo.Var(m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    m.capex = pyo.Var(m.K, domain=pyo.NonNegativeReals)
    m.total_capex = pyo.Var(domain=pyo.NonNegativeReals)
    m.TPC = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.mantainance_costs = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.administrative_costs = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.total_opex = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.opex = pyo.Var(m.K, m.T, domain=pyo.NonNegativeReals)
    
    m.TTC = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.TTC_1 = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.TTC_2 = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.TTC_3 = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    
    m.OWK = pyo.Var(m.K, m.P, m.T, m.ST, domain=pyo.NonNegativeReals)
    m.WC= pyo.Var(m.K, m.P, m.T, domain=pyo.NonNegativeReals)
    m.TWC = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    
    m.npc = pyo.Var(domain=pyo.NonNegativeReals)
    
    
    ####################CONSTRAINTS####################

    m.constraints = pyo.ConstraintList()
        
    #Storage Facilty Allocation
    for k in m.K:
        m.constraints.add(sum(m.w[k,st] for st in m.ST) == m.v[k])
        m.constraints.add(sum(m.v[k] for k in m.K) <= 3)
        for st in m.ST:
            m.constraints.add(m.w[k,st] <= m.v[k])
    
    #Demand Supply
    for t in m.T:
        for j in m.J:
            for p in m.P:
                m.constraints.add(sum(m.QKJ[k,j,p,t] for k in m.K) >= dem.loc[(j,p,t),'demand'])
                
    #Mass Balance
    for k in m.K:
        for t in m.T:
            m.constraints.add(m.QK_total[k,t] == sum(m.QK[k,p,t] for p in m.P))
            
            m.constraints.add(m.QK_total[k,t] >= sum(df_facilities_1.loc[st,'low']*m.w[k,st] for st in m.ST))
            
            m.constraints.add(m.QK_total[k,t] <= sum(df_facilities_1.loc[st,'up']*m.w[k,st] for st in m.ST))
            
            
            for p in m.P:
                m.constraints.add(m.QK[k,p,t] == sum(m.QIK[i,k,p,t] for i in m.I) + sum(m.QKK[kk,k,p,t] for kk in m.K))
                
                m.constraints.add(m.flow_in[k,p,t] == sum(m.QIK[i,k,p,t] for i in m.I) + sum(m.QKK[kk,k,p,t] for kk in m.K))
                
                m.constraints.add(m.flow_out[k,p,t] == sum(m.QKK[k,kk,p,t] for kk in m.K) + sum(m.QKJ[k,j,p,t] for j in m.J))
                
                m.constraints.add(m.flow_in[k,p,t] == m.flow_out[k,p,t])
                
                m.constraints.add(m.QK[k,p,t] >= sum(df_facilities_2.loc[(st,p), 'low']*m.wq[k,p,st] for st in m.ST))
                
                m.constraints.add(m.QK[k,p,t] <= sum(df_facilities_2.loc[(st,p), 'up']*m.wq[k,p,st] for st in m.ST))

                for st in m.ST:
                    m.constraints.add( m.wq[k,p,st] <= m.v[k])
                    m.constraints.add( m.wq[k,p,st] <= m.w[k,st])
                    m.constraints.add( m.wq[k,p,st] <= m.q[k,p])
                    m.constraints.add( m.wq[k,p,st] <= m.q[k,p] + m.w[k,st] - 1)

       
    #Primary Links
    for i in m.I:
        for k in m.K:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.x[i,k,p,t] <= m.v[k])
                    m.constraints.add(m.QIK[i,k,p,t] <= big_m*m.v[k])
                    m.constraints.add(m.QIK[i,k,p,t] <= big_m*m.q[k,p])
                    m.constraints.add(m.QIK[i,k,p,t] <= df_suppliers_2.loc[(i,p),'up']*m.x[i,k,p,t])
                    
    for i in m.I:
        for p in m.P:
            for t in m.T:
                m.constraints.add(sum(m.QIK[i,k,p,t] for k in m.K) <= df_suppliers_1.loc[i,'up'])
    
    #Intermediate links
    for k in m.K:
        for kk in m.K:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.y[k,kk,p,t] <= m.v[k])
                    m.constraints.add(m.y[k,kk,p,t] <= m.v[kk])
                    m.constraints.add(m.y[k,kk,p,t] <= m.q[k,p])
                    m.constraints.add(m.y[k,kk,p,t] <= m.q[kk,p])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.v[k])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.v[kk])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.q[k,p])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.q[kk,p])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.y[k,kk,p,t])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.y[kk,k,p,t])
                                   
    #Final Links
    for k in m.K:
        for j in m.J:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.z[k,j,p,t] <= m.v[k])
                    m.constraints.add(m.z[k,j,p,t] <= m.q[k,p])
                    
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.v[k])
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.q[k,p])
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.z[k,j,p,t])
    
    
    #Investments
    def r_capex_a(m, k, t):
        return m.capex_products[k, p, t] == df_facilities_2.loc[(st,p),'allocation_cost']*(m.q[k,p] + m.w[k,st] - 1)
    m.r_capex_a = pyo.Constraint(m.K, m.T, rule = r_capex_a)
    
    
    def r_capex_b(m, k):
        return m.capex[k] == sum(m.w[k,st]*df_facilities_1.loc[st,'capex'] for st in m.ST)
    m.r_capex_b = pyo.Constraint(m.K, rule = r_capex_b)
    
    
    def r_total_capex(m):
        return m.total_capex == sum(m.capex[k] for k in m.K)
    m.r_total_capex = pyo.Constraint(rule=r_total_capex)
    
    
    def r_purchase_cost(m, i, p, t):
        return m.TPC[t] == sum(m.QIK[i,k,p,t]*df_suppliers_2.loc[(i,p),'purchase_cost'] for i in m.I for k in m.K for p in m.P)
    m.r_purchase_cost = pyo.Constraint(m.I, m.P, m.T, rule=r_purchase_cost)
    
    
    #Operational Costs
    for t in m.T:
        m.constraints.add(m.mantainance_costs[t] == sum(m.q[k,p]*df_products.loc[p,'mantainance_cost'] for k in m.K for p in m.P))
        m.constraints.add(m.administrative_costs[t] == sum(m.w[k,st]*df_facilities_1.loc[st,'fixed_cost'] for k in m.K for p in m.P))
        m.constraints.add(m.total_opex[t] == sum(m.opex[k,t] for k in m.K))
        for k in m.K:
            m.constraints.add(m.opex[k,t] == sum(m.QKST[k,p,st,t]*df_facilities_2.loc[(st,p),'handling_cost'] for p in m.P for st in m.ST))
            for st in m.ST:
                for p in m.P:
                    m.constraints.add(m.QK[k,p,t] - df_facilities_2.loc[(st,p),'up']*(1 - m.w[k,st]) <= m.QKST[k,p,st,t] )
                    m.constraints.add(m.QKST[k,p,st,t] <= df_facilities_2.loc[(st,p),'up']*m.w[k,st])
    
    ## Transportation Costs
    for t in m.T:
        m.constraints.add( m.TTC[t] == m.TTC_1[t] + m.TTC_2[t] + m.TTC_3[t] )
        
        m.constraints.add(m.TTC_1[t] == sum(dist_1[(i,k)]*m.QIK[i,k,p,t]*df_products.loc[p,'t_primary_cost'] \
                         for i in m.I for k in m.K for p in m.P))
        
        m.constraints.add(m.TTC_2[t] == sum(dist_2[(k,kk)]*m.QKK[k,kk,p,t]*df_products.loc[ p,'t_intermediate_cost'] \
                         for k in m.K for kk in m.K for p in m.P))
        
        m.constraints.add(m.TTC_3[t] == sum(dist_3[(k,j)]*m.QKJ[k,j,p,t]*df_products.loc[p,'t_final_cost'] \
                         for k in m.K for j in m.J for p in m.P)) 
          
    #Inventory Costs    
#     for p in m.P:
#         m.constraints.add(m.TAS[p] == sum(order_size[(p,st),'order_size']/2 * m.wq[k,p,st] for k in m.K for st in m.ST))

    #Waiting Costs
    def r_waiting_time(m,k,p,t,st):
        return m.OWK[k,p,t,st] == sum(m.QKJ[k,j,p,t]*(df_facilities_2.loc[(st,p),'processing_time'] + dist_3[(k,j)]/30)*df_products.loc[p,'waiting_cost'] for j in m.J)
    m.r_waiting_time = pyo.Constraint(m.K, m.P, m.T, m.ST, rule=r_waiting_time)
    
    def r_waiting_cost(m,k,p,t,st):
        return m.WC[k,p,t] >= m.OWK[k,p,t,st] - big_m*(1-m.w[k,st])
    m.r_waiting_cost = pyo.Constraint(m.K, m.P, m.T, m.ST, rule=r_waiting_cost)
    
    for t in m.T:
        m.constraints.add(m.TWC[t] == sum(m.WC[k,p,t] for k in m.K for p in m.P))
        
    def r_npc(m):
        return m.npc == m.total_capex + sum(((m.TPC[t] + m.TTC[t] + m.total_opex[t] + \
                                            m.mantainance_costs[t] + \
                                            m.administrative_costs[t] + m.TWC[t])/(1+0.15)**float(t.strip()[1])) \
                                            for t in m.T)
    # + m.TSC[t]
    
    m.r_npc = pyo.Constraint(rule=r_npc)
    def OBJ(m):
        return m.npc
    m.OBJ = pyo.Objective(rule = OBJ, sense = pyo.minimize)
    
    return m

build_model()

In [21]:
# WARNING! This cell excecution time takes around 55 minutes.

m = build_model()

opt = pyo.SolverFactory("glpk",tee=True)
results = opt.solve(m)
results.write()



# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17953.7403027759
  Upper bound: 17953.7403027759
  Number of objectives: 1
  Number of constraints: 71619
  Number of variables: 21407
  Number of nonzeros: 221210
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 6271
      Number of created subproblems: 6271
  Error rc: 0
  Time: 3200.092708826065
# ----------------------------------------------------------
#   Solution Information


In [22]:
m = build_model()
opt = pyo.SolverFactory("../cbc-win64/cbc", tee=True)
results = opt.solve(m)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 71618
  Number of variables: 21406
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CBC 2.10.3 optimal, objective 17953.740302776; 22 nodes, 20699 iterations, 271.574 seconds
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 257.79894948005676
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number o

In [23]:
m = build_model()
opt = pyo.SolverFactory("cplex", tee=True)
results = opt.solve(m)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpday5589h
  Lower bound: 17953.740302775936
  Upper bound: 17953.740302775936
  Number of objectives: 1
  Number of constraints: 71619
  Number of variables: 21407
  Number of nonzeros: 221210
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 3.91
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a Objective = 1.7953740303e+04
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Tim

In [40]:
print("Print values for all variables")
for v in m.component_data_objects(pyo.Var):
    if v.value:
        if v.value > 0:
            print(str(v), v.value)

Print values for all variables
v[k1] 1.0
v[k3] 1.0
v[k7] 1.0
w[k1,st1] 1.0
w[k3,st1] 1.0
w[k7,st1] 1.0
wq[k1,p0,st1] 1.0
wq[k1,p1,st1] 1.0
wq[k1,p2,st1] 1.0
wq[k1,p3,st1] 1.0
wq[k1,p4,st1] 1.0
wq[k1,p5,st1] 1.0
wq[k3,p0,st1] 1.0
wq[k3,p1,st1] 1.0
wq[k3,p2,st1] 1.0
wq[k3,p3,st1] 1.0
wq[k3,p4,st1] 1.0
wq[k3,p5,st1] 1.0
wq[k7,p0,st1] 1.0
wq[k7,p1,st1] 1.0
wq[k7,p2,st1] 1.0
wq[k7,p3,st1] 1.0
wq[k7,p4,st1] 1.0
wq[k7,p5,st1] 1.0
q[k0,p0] 1.0
q[k0,p1] 1.0
q[k0,p2] 1.0
q[k0,p3] 1.0
q[k0,p4] 1.0
q[k0,p5] 1.0
q[k1,p0] 1.0
q[k1,p1] 1.0
q[k1,p2] 1.0
q[k1,p3] 1.0
q[k1,p4] 1.0
q[k1,p5] 1.0
q[k2,p0] 1.0
q[k2,p1] 1.0
q[k2,p2] 1.0
q[k2,p3] 1.0
q[k2,p4] 1.0
q[k2,p5] 1.0
q[k3,p0] 1.0
q[k3,p1] 1.0
q[k3,p2] 1.0
q[k3,p3] 1.0
q[k3,p4] 1.0
q[k3,p5] 1.0
q[k4,p0] 1.0
q[k4,p1] 1.0
q[k4,p2] 1.0
q[k4,p3] 1.0
q[k4,p4] 1.0
q[k4,p5] 1.0
q[k5,p0] 1.0
q[k5,p1] 1.0
q[k5,p2] 1.0
q[k5,p3] 1.0
q[k5,p4] 1.0
q[k5,p5] 1.0
q[k6,p0] 1.0
q[k6,p1] 1.0
q[k6,p2] 1.0
q[k6,p3] 1.0
q[k6,p4] 1.0
q[k6,p5] 1.0
q[k7,p0] 1.0
q[k7,p1] 1.0
q